In [12]:
# Warnings configuration
import warnings
warnings.filterwarnings('ignore')

# General Libraries
import tensorflow as tf
import numpy as np

# Neural Network Components
from tensorflow import keras
from keras import layers
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K

In [13]:
pretrained_model = keras.models.load_model('my_model3.keras')
data_dir = '/tf/keras_neural_network/Mis_Tests/Faces'

In [14]:
h = 109
w = 109
image_size = (h, w,3)
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    labels='inferred',
    label_mode='categorical', 
    image_size=image_size[:2],
    interpolation='bilinear',
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    labels='inferred',
    label_mode='categorical', 
    image_size=image_size[:2],
    interpolation='bilinear',
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    
)

Found 9711 files belonging to 2 classes.
Using 7769 files for training.
Found 9711 files belonging to 2 classes.
Using 1942 files for validation.


In [15]:
warnings.filterwarnings('always')

In [16]:
base_model = keras.Model(
    inputs=pretrained_model.input,
    outputs=pretrained_model.output
)
base_model.trainable = True

In [17]:
base_model.compile(
    loss="binary_crossentropy", 
    optimizer=eval(f"keras.optimizers.{'Adam'}(learning_rate=0.0001)"), 
    metrics=["binary_accuracy"]
)

In [18]:
base_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 109, 109, 3)]     0         
                                                                 
 model (Functional)          (None, 1024)              7041600   
                                                                 
 dense_Face_ID (Dense)       (None, 52)                53300     
                                                                 
 batch_normalization_1 (Bat  (None, 52)                208       
 chNormalization)                                                
                                                                 
 dropout_1 (Dropout)         (None, 52)                0         
                                                                 
 Face_ID_layer (Dense)       (None, 2)                 106       
                                                           

In [19]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_binary_accuracy',
    min_delta=0.001,
    patience=3, 
    restore_best_weights=True,
    mode="auto",
    verbose=1,
    baseline=None
)

In [20]:
ModelCheckpoint = keras.callbacks.ModelCheckpoint(
    '/tf/Face_ID1/my_model4.keras',
    monitor='val_binary_accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch",
    initial_value_threshold=None,
)

In [21]:
history = base_model.fit(
    train_ds,
    epochs=20, 
    batch_size=batch_size,
    verbose=1,
    validation_data=val_ds,
    callbacks=[early_stopping,
              ModelCheckpoint]
)

Epoch 1/20
243/243 [==============================] - 68s 97ms/step - loss: 1.3882 - binary_accuracy: 0.8457 - val_loss: 0.7028 - val_binary_accuracy: 0.8154
Epoch 2/20
243/243 [==============================] - 21s 87ms/step - loss: 0.4951 - binary_accuracy: 0.9405 - val_loss: 0.1762 - val_binary_accuracy: 0.9858
Epoch 3/20
243/243 [==============================] - 22s 91ms/step - loss: 0.3092 - binary_accuracy: 0.9643 - val_loss: 0.1453 - val_binary_accuracy: 0.9897
Epoch 4/20
243/243 [==============================] - 20s 81ms/step - loss: 0.2049 - binary_accuracy: 0.9790 - val_loss: 0.1042 - val_binary_accuracy: 0.9961
Epoch 5/20
243/243 [==============================] - 20s 83ms/step - loss: 0.1703 - binary_accuracy: 0.9828 - val_loss: 0.1119 - val_binary_accuracy: 0.9949
Epoch 6/20
243/243 [==============================] - 18s 74ms/step - loss: 0.1392 - binary_accuracy: 0.9867 - val_loss: 0.0988 - val_binary_accuracy: 0.9954
Epoch 7/20
243/243 [==============================] 

In [22]:
base_model.save('my_model4.keras')